In [ ]:
import pandas as pd
import numpy as np

species = 'mansoni-highAdultBurden'
NT = '0'
targetPrev = 70
treatmentInterval = 0.5

effArray = [40, 86]
longLastArray = [3, 5, 10, 15]

case = [None]*(len(effArray)+len(longLastArray)+2)
label = [None]*(len(effArray)+len(longLastArray)+2)
label1line = [None]*(len(effArray)+len(longLastArray)+2)


case[0] = species + '-'+str(targetPrev)+'prev-' + NT + 'NT-'+str(int(1/treatmentInterval))+'pyr-baseline'
label[0] = 'Baseline'
label1line[0] = 'Baseline'
i=0
for j in effArray:
    i = i+ 1
    case[i] = species + '-'+str(targetPrev)+'prev-' + NT + 'NT-'+str(int(1/treatmentInterval))+'pyr-juvenileDrugEfficacy_'+str(j)
    label[i] = str(j)+'% juvenile \n efficacy'
    label1line[i] = str(j)+'% juvenile efficacy'

for j in longLastArray:
    i = i+ 1
    case[i] = species + '-'+str(targetPrev)+'prev-' + NT + 'NT-'+str(int(1/treatmentInterval))+'pyr-longLastingEfficacy_'+str(j)
    label[i] = str(j)+' week \n efficacy'
    label1line[i] = str(j)+' week efficacy'

case[i+1] = species + '-'+str(targetPrev)+'prev-' + NT + 'NT-'+str(int(1/treatmentInterval))+'pyr-99efficacy'
label[i+1] = '99% adult \n efficacy'
label1line[i+1] = '99% adult efficacy'

df = np.zeros((511,26,len(case)))
eggCounts = np.zeros((511,500,len(case)))
wormCounts = np.zeros((511,500,len(case)))
params = [None]*len(case)


for i in range(len(case)):
    df[:,:,i] = pd.read_json('Data_output/' + case[i] + '_df_results.json')
    eggCounts[:,:,i] = np.load('Data_output/' +  case[i] + '_egg_results.npy')
    wormCounts[:,:,i] = np.load('Data_output/' +  case[i] + '_worm_results.npy')
    params[i] = np.load('Data_output/'  + case[i] + '_params.npy',allow_pickle='TRUE').item()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style

plt.style.use('ggplot')


plt.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,22,0]/df[0,22,0],label='Adults')
plt.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,23,0]/df[0,23,0],label='Juveniles')
plt.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,24,0]/df[0,24,0],label='Free living')


plt.ylabel('Prevalence')
plt.xlabel('Year')
plt.legend(ncol=1)

plt.show()     

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
plt.style.use('ggplot')


ax1.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,4,0],label='SAC', color='r')
ax1.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,7,0],label='Adults', color='g')
ax1.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,13,0],label='SAC high intensity',linestyle='dotted', color='r')
ax1.plot(df[:,0,0] - np.min(df[:,0,0]) ,df[:,16,0],label='Adults high intensity',linestyle='dotted', color='g')

ax2.plot(df[:,0,1] - np.min(df[:,0,1]) ,df[:,4,1],label='SAC', color='r')
ax2.plot(df[:,0,1] - np.min(df[:,0,1]) ,df[:,7,1],label='Adults', color='g')
ax2.plot(df[:,0,1] - np.min(df[:,0,1]) ,df[:,13,1],label='SAC high intensity',linestyle='dotted', color='r')
ax2.plot(df[:,0,1] - np.min(df[:,0,1]) ,df[:,16,1],label='Adults high intensity',linestyle='dotted', color='g')

plt.ylabel('Prevalence')
plt.xlabel('Year')
plt.legend(ncol=2)

plt.show()     

In [ ]:
elimination = np.zeros((np.size(wormCounts,axis=0),len(case)))
elim90Time = np.zeros(len(case))
elim20Prob = np.zeros(len(case))
plt.figure(figsize=(5,4))

for i in range(1,len(case)):
    elimination[:,i] = np.sum(wormCounts[:,:,i]<=0.0,axis=1)/np.size(wormCounts[:,:,i],1)

    plt.plot(df[:,0,i] - np.min(df[:,0,i]) ,elimination[:,i]*100, label=label[i])

    elim90Time[i] = np.ceil((np.max(np.asarray(elimination[:,i]<0.9).nonzero())+2)*(params[i]['outTimings'][1]-params[i]['outTimings'][0]))
    elim20Prob[i] = elimination[np.where(df[:,0,i]==params[i]['chemoTimings'][0]+20)[0],i]


elimination[:,0] = np.sum(wormCounts[:,:,0]<=0.0,axis=1)/np.size(wormCounts[:,:,0],1)
elim90Time[0] = np.ceil((np.max(np.asarray(elimination[:,0]<0.9).nonzero())+2)*(params[0]['outTimings'][1]-params[0]['outTimings'][0]))
elim20Prob[0] = elimination[np.where(df[:,0,0]==params[0]['chemoTimings'][0]+20)[0],0]
plt.plot(df[:,0,0] - np.min(df[:,0,0]) ,elimination[:,0]*100, label=label[0], color='black')

plt.ylabel('Probability of elimination (%)')
plt.xlabel('Time since start of MDA (years)')

plt.xticks(np.linspace(1,31,7),np.linspace(0,30,7))
plt.xlim(0,31)
plt.show()

[elim90Time, elim20Prob*100]

In [ ]:
pLevels = [0.7, 0.5, 0.25, 0.1, 0.01]

elimTime=np.zeros((np.size(wormCounts,axis=1),len(case)))
prevTime=np.zeros((np.size(wormCounts,axis=1),len(pLevels),len(case)))
thresholdtimes=np.zeros((len(pLevels),len(case)))

for i in range(len(case)):
    elimTime[:,i] = np.size(wormCounts[:,:,i],axis=0) - (np.argmax(wormCounts[::-1,:,i]>0,axis=0)-1) 
    for j in range(len(pLevels)):
        prevTime[:,j,i] = np.size(wormCounts[:,:,i],axis=0) - (np.argmax(wormCounts[::-1,:,i]>pLevels[j],axis=0)-1)
    prevTime[prevTime==len(wormCounts)+1]=np.nan
    thresholdtimes[:,i] = np.nanmean((elimTime[:,i]-prevTime[:,:,i].T).T*(params[i]['outTimings'][1]-params[i]['outTimings'][0]) ,axis=0)

plt.figure(figsize=(12,6))

for i in range(len(pLevels)):
    plt.bar(label,thresholdtimes[i,:])
    for j in range(len(case)):
        plt.text(j-0.39,thresholdtimes[i,j]-0.0015*np.max(elimTime), str(int(pLevels[i]*100))+'%')

for j in range(len(case)):
    for i in range(len(thresholdtimes)-1):
        plt.text(j,thresholdtimes[i+1,j]+(thresholdtimes[i,j]-thresholdtimes[i+1,j])/2, str(round(thresholdtimes[i,j]-thresholdtimes[i+1,j],1)),
                horizontalalignment='center', verticalalignment='center')

    plt.text(j,(thresholdtimes[-1,j])/2, str(round(thresholdtimes[-1,j],1)),
                horizontalalignment='center', verticalalignment='center')

plt.ylabel('Time until elimination (years)')
plt.show()

In [ ]:
import numpy as np

P1 = wormCounts[:,:,0]  * 100

P2 = wormCounts[:,:,3] * 100

plt.plot(df[:,0,0] - np.min(df[:,0,0]) ,np.mean(P1,axis=1),label  = 'Baseline case')
plt.fill_between(df[:,0,0] - np.min(df[:,0,0]) , np.percentile(P1,2.5,axis=1),
                     np.percentile(P1,97.5,axis=1), alpha=0.25)

plt.plot(df[:,0,1] - np.min(df[:,0,1]) ,np.mean(P2,axis=1), label = 'Improved case')
plt.fill_between(df[:,0,1] - np.min(df[:,0,1]) , np.percentile(P2,2.5,axis=1),
                     np.percentile(P2,97.5,axis=1), alpha=0.25)
 
plt.ylabel('Prevalence (%)')
plt.xlabel('Year')
plt.legend(ncol=1)

plt.show()      

In [ ]:
import numpy as np

plt.figure(figsize=(5,4))


for i in range(1,len(case)):
    plt.plot(df[:,0,i] - np.min(df[:,0,i]) ,np.mean( wormCounts[:,:,i] * 100,axis=1),label = label1line[i])

plt.plot(df[:,0,0] - np.min(df[:,0,0]) ,np.mean( wormCounts[:,:,0] * 100,axis=1),label = label1line[0],color='black')
plt.ylabel('Prevalence (%)')
plt.xlabel('Time since start of MDA (years)')
plt.legend(ncol=1)

plt.xticks(np.linspace(1,31,7),np.linspace(0,30,7))
plt.xlim(0,31)
plt.show()      

In [ ]:
fig= plt.figure(figsize=(12, 10))

gs = fig.add_gridspec(2,2, height_ratios=[1,1.5])
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])
axes = [ax1, ax2, ax3]


for i in range(1,len(case)):
    ax1.plot(df[:,0,i] - np.min(df[:,0,i]) ,np.mean( wormCounts[:,:,i] * 100,axis=1),label = label1line[i])


ax1.plot(df[:,0,0] - np.min(df[:,0,0]) ,np.mean( wormCounts[:,:,0] * 100,axis=1),label = label[0],color='black')
ax1.set_ylabel('Prevalence (%)')
ax1.set_xlabel('Time since start of MDA (years)')
ax1.legend(ncol=1)

# plt.xticks(np.linspace(1,15,8),np.linspace(0,14,8))
# plt.xlim(0,15)

################################################################################################################################0
for i in range(1,len(case)):
    ax2.plot(df[:,0,i] - np.min(df[:,0,i]) ,elimination[:,i]*100, label=label[i])
ax2.plot(df[:,0,0] - np.min(df[:,0,0]) ,elimination[:,0]*100, label=label[0], color='black')

ax2.set_ylabel('Probability of elimination (%)')
ax2.set_xlabel('Time since start of MDA (years)')

# plt.xticks(np.linspace(1,15,8),np.linspace(0,14,8))

#################################################################################################################################

for i in range(len(pLevels)):
    ax3.bar(label,thresholdtimes[i,:])
    for j in range(len(case)):
        ax3.text(j-0.39,thresholdtimes[i,j]-0.002*np.max(elimTime), str(int(pLevels[i]*100))+'%')

for j in range(len(case)):
    for i in range(len(thresholdtimes)-1):
        ax3.text(j,thresholdtimes[i+1,j]+(thresholdtimes[i,j]-thresholdtimes[i+1,j])/2, str(round(thresholdtimes[i,j]-thresholdtimes[i+1,j],1)),
                horizontalalignment='center', verticalalignment='center')

    ax3.text(j,(thresholdtimes[-1,j])/2, str(round(thresholdtimes[-1,j],1)),
                horizontalalignment='center', verticalalignment='center')

ax3.set_xticklabels(label,rotation=0)#, ha='right',rotation_mode='anchor')
ax3.set_ylabel('Time until elimination (years)')

ax1.text(-0.1, 1, 'A', fontsize='18',weight="bold", horizontalalignment='right',verticalalignment='bottom',transform=ax1.transAxes)
ax2.text(-0.1, 1, 'B', fontsize='18',weight="bold", horizontalalignment='right',verticalalignment='bottom',transform=ax2.transAxes)
ax3.text(-0.05, 1, 'C', fontsize='18',weight="bold", horizontalalignment='right',verticalalignment='bottom',transform=ax3.transAxes)

xlimit = 15
ax1.set_xticks(np.linspace(1,xlimit+1,6),np.linspace(0,xlimit,6))
ax2.set_xticks(np.linspace(1,xlimit+1,6),np.linspace(0,xlimit,6))
ax1.set_xlim(0,xlimit+1)
ax2.set_xlim(1,xlimit+1)


# plt.savefig('Figures/Fig1.png', bbox_inches='tight',dpi=300)
# plt.savefig('Figures/Fig1.tiff', bbox_inches='tight',dpi=300)
# plt.savefig('Figures/Fig1.eps', bbox_inches='tight',dpi=300)

plt.show()